In [5]:
solver = 'appsi_highs'

import pyomo.environ as pyo

SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(),f'solver {solver} is not available'

In [17]:
model = pyo.ConcreteModel('Production Planning: Version 1')
model.display()

Model 'Production Planning: Version 1'

  Variables:
    None

  Objectives:
    None

  Constraints:
    None


In [21]:
model.x_M = pyo.Var(bounds = (0,None))
model.x_A = pyo.Var(bounds = (0,80))
model.x_B = pyo.Var(bounds = (0,100))

model.y_U = pyo.Var(bounds = (0,40))
model.y_V = pyo.Var(bounds = (0,None))

model.display()

Model 'Production Planning: Version 1'

  Variables:
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    80 : False :  True :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   100 : False :  True :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    40 : False :  True :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

  Objectives:
    None

  Constraints:
    None


In [23]:
# create expression
model.revenue = 270*model.y_U + 210*model.y_V
model.cost = 10*model.x_M + 50*model.x_A + 40*model.x_B

print(model.revenue)
print(model.cost)

270*y_U + 210*y_V
10*x_M + 50*x_A + 40*x_B


In [37]:
model.profit = pyo.Objective(expr=model.revenue - model.cost, sense=pyo.maximize)

'pyomo.core.base.objective.ScalarObjective'>) on block 'Production Planning:
Version 1' with a new Component (type=<class
'pyomo.core.base.objective.ScalarObjective'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [39]:
model.raw_materials = pyo.Constraint(expr = 10*model.y_U + 9*model.y_V <= model.x_M)
model.labor_A = pyo.Constraint(expr = 1* model.y_U + 1* model.y_V <= model.x_A)
model.labor_B = pyo.Constraint(expr = 2* model.y_U + 1* model.y_V <= model.x_B)

In [41]:
model.display()

Model 'Production Planning: Version 1'

  Variables:
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    80 : False :  True :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   100 : False :  True :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    40 : False :  True :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

  Objectives:
    profit : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: y_U
        (object: <class 'pyomo.core.base.var.ScalarVar'>)
    No value for 

In [43]:
results = SOLVER.solve(model, tee = True)

Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+01, 3e+02]
  Bound  [4e+01, 1e+02]
  RHS    [0e+00, 0e+00]
Presolving model
2 rows, 4 cols, 6 nonzeros  0s
2 rows, 4 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-1); columns 4(-1); elements 6(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999953995e+02 Ph1: 2(4); Du: 3(210) 0s
          3    -2.6000000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  2.6000000000e+03
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [45]:
model.pprint()

5 Var Declarations
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  80.0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 100.0 :   100 : False : False :  Reals
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 740.0 :  None : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False :  Reals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)

3 Constraint Dec

In [49]:
model.profit.pprint()

profit : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)


In [51]:
pyo.value(model.profit)

2600.0

In [53]:
pyo.value(model.revenue)

18000.0

In [57]:
model.x_A()

80.0

# data driven model

In [1]:
products = {
    "U": {"price": 270, "demand": 40},
    "V": {"price": 210, "demand": None},
}

resources = {
    "M": {"price": 10, "available": None},
    "labor A": {"price": 50, "available": 80},
    "labor B": {"price": 40, "available": 100},
}

processes = {
    "U": {"M": 10, "labor A": 1, "labor B": 2},
    "V": {"M": 9, "labor A": 1, "labor B": 1},
}


In [7]:
model = pyo.ConcreteModel('Production Planning : version 2')

In [9]:
model.PRODUCTS = pyo.Set(initialize = products.keys())
model.RESOURCES = pyo.Set(initialize = resources.keys())

In [11]:
# parameter for bounds
@model.Param(model.PRODUCTS, domain=pyo.Any)
def demand(model, product):
    return products[product]["demand"]


@model.Param(model.RESOURCES, domain=pyo.Any)
def available(model, resource):
    return resources[resource]["available"]


# parameter with price coefficients
@model.Param(model.PRODUCTS)
def cp(model, product):
    return products[product]["price"]


@model.Param(model.RESOURCES)
def cr(model, resource):
    return resources[resource]["price"]


# process parameters: a[r,p]
@model.Param(model.RESOURCES, model.PRODUCTS)
def a(model, resource, product):
    return processes[product][resource]

In [13]:
model.x = pyo.Var(model.RESOURCES, bounds = lambda model, resource:(0, model.available[resource]))
model.y = pyo.Var(model.PRODUCTS, bounds = lambda model, product: (0, model.demand[product]))

In [15]:
model.revenue = pyo.quicksum(
        model.cp[product]*model.y[product] for product in model.PRODUCTS
)

model.cost = pyo.quicksum(
        model.cr[resource]*model.x[resource] for resource in model.RESOURCES
)

# Create objective
@model.Objective(sense=pyo.maximize)
def profit(model):
    return model.revenue-model.cost



In [17]:
# CREATE CONSTRIANTS
@model.Constraint(model.RESOURCES)
def material_used(model, resource):
    return (pyo.quicksum(
                model.a[resource, product] * model.y[product] for product in model.PRODUCTS
            ) <= model.x[resource]
           )

In [19]:
model.pprint()

2 Set Declarations
    PRODUCTS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'U', 'V'}
    RESOURCES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'M', 'labor A', 'labor B'}

5 Param Declarations
    a : Size=6, Index=RESOURCES*PRODUCTS, Domain=Any, Default=None, Mutable=False
        Key              : Value
              ('M', 'U') :    10
              ('M', 'V') :     9
        ('labor A', 'U') :     1
        ('labor A', 'V') :     1
        ('labor B', 'U') :     2
        ('labor B', 'V') :     1
    available : Size=3, Index=RESOURCES, Domain=Any, Default=None, Mutable=False
        Key     : Value
              M :  None
        labor A :    80
        labor B :   100
    cp : Size=2, Index=PRODUCTS, Domain=Any, Default=None, Mutable=False
        Key : Value
          U :   270
          V :   210
    cr : Size=3, I

In [21]:
SOLVER.solve(model)

{'Problem': [{'Lower bound': 2600.0, 'Upper bound': 2600.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [23]:
print(f'Profit: {pyo.value(model.profit)}')
print("\nProduction Report")
for product in model.PRODUCTS:
    print(f"{product} produced = {pyo.value(model.y[product])}")
print("\nResource Report")
for resource in model.RESOURCES:
    print(f"{resource} consumed = {pyo.value(model.x[resource])}")

Profit: 2600.0

Production Report
U produced = 20.0
V produced = 60.0

Resource Report
M consumed = 740.0
labor A consumed = 80.0
labor B consumed = 100.0


# Using OOPs

In [28]:
class ProductionModel(pyo.ConcreteModel):
    """
    A class representing a production model using Pyomo
    """
    def __init__(self, products, resources, processes):

        super().__init__("Production Model")
        self.products = products
        self.resources = resources
        self.processes = processes

        # flag to moniter the solution status
        self.solved = False

    def build_model(self):
        model = self.model()
        model.PRODUCTS = self.products.keys()
        model.RESOURCES = self.resources.keys()

        model.x = pyo.Var(
            model.RESOURCES, bounds = lambda model, resource:(0, self.resources[resource]['available'])
        )
        
        model.y = pyo.Var(
            model.PRODUCTS, bounds= lambda model, resource:(0, self.products[product]['demand'])
        )

        model.revenue = pyo.quicksum(
            self.products[product]['price']*model.y[product] for product in model.products
        )
        
        model.cost = pyo.quicksum(
            self.resources[resource]['price']*model.x[resource] for resource in model.resources
        )

        # create objective
        @model.Objective(sense = pyo.maximize)
        def profit(model):
            return model.revenue - model.cost
        # add constraints
        @model.Constraint(model.RESOURCES)
        def materials_used(model, resource):
            return (
                pyo.quicksum(
                    self.processes[product][resource] * model.y[product] for product in model.PRODUCTS
                ) <= model.x[resource]
            )
    def solve(self, solver = SOLVER):
        self.build_model()
        solver.solve(self)
        self.solved = True
    def report(self):
        if not self.solved:
            self.solve(SOLVER)
        print(f"Profit = {pyo.value(self.profit)}")
        print("\nProduction Report")
        for product in m.PRODUCTS:
            print(f" {product}  produced =  {pyo.value(self.y[product])}")
        print("\nResource Report")
        for resource in m.RESOURCES:
            print(f" {resource} consumed = {pyo.value(self.x[resource])}")

m = ProductionModel(products, resources, processes)

In [30]:
m.report()

Profit = 2600.0

Production Report
 U  produced =  20.0
 V  produced =  60.0

Resource Report
 M consumed = 740.0
 labor A consumed = 80.0
 labor B consumed = 100.0
